In [86]:
import glob
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
import shutil
import seaborn as sns
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline  

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/Users/joaoneves/anaconda3/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'


In [87]:
london_coordinates = np.array([51.5073509, -0.12775829999998223])
np.random.seed(0)
np.random.uniform(-0.1,0.1)

0.009762700785464956

In [88]:
def generate_gps_coordinates(center, radius_dists, n=1000):
    result = []
    for i in range(n):
        point = np.add(center, np.array([dist(-0.1,0.1) for dist in radius_dists]))
        result.append(point)
    return np.array(result)

In [89]:
generate_gps_coordinates(london_coordinates, (np.random.uniform, np.random.uniform))

array([[ 51.55038877,  -0.10720562],
       [ 51.51632754,  -0.14302734],
       [ 51.53652972,  -0.14024086],
       ..., 
       [ 51.56560721,  -0.08351868],
       [ 51.50337246,  -0.09898549],
       [ 51.50770553,  -0.06545461]])

In [138]:
data_dir = "/Users/joaoneves/Documents/demo-siemens/data/transactions_data/PS_20174392719_1491204439457_log.csv"
transactions_df = pd.read_csv(data_dir)
n = len(transactions_df)
n

6362620


In [139]:
transactions_df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [140]:
gps_coordinates = generate_gps_coordinates(london_coordinates, (np.random.uniform, np.random.uniform), n=len(transactions_df))

In [141]:
training_path = "/Users/joaoneves/Documents/demo-iot-transactions/data/transactions_data/transactions_with_gps.csv"
transactions_df["gps_latitude"] = pd.Series(gps_coordinates[:,0])
transactions_df["gps_longitude"] = pd.Series(gps_coordinates[:,1])
transactions_df["isFraud"] = transactions_df["isFraud"].astype("category")
transactions_df["id"] = transactions_df.index
transactions_df.to_csv(training_path, index=False)

In [142]:
# transactions_df["isFraud"].describe()
transactions_df["isFraud"].value_counts()

0    6354407
1    8213   
Name: isFraud, dtype: int64

In [143]:
tests = np.array_split(transactions_df, 100000)

In [155]:
test_path

'/Users/joaoneves/Documents/demo-iot-transactions/data/transactions'

In [156]:
test_files_path = os.path.join(test_path, "transactions*.csv")
test_files = glob.glob(test_files_path)

In [157]:
test_files.index('/Users/joaoneves/Documents/demo-iot-transactions/data/transactions/transactions_5.csv')

44445

In [146]:
test_path = "/Users/joaoneves/Documents/demo-iot-transactions/data/transactions"

for file in test_files:
    os.remove(file)

for i, df in enumerate(tests):
    df.to_csv(os.path.join(test_path, "transactions_{}.csv".format(i)), index=False)

In [160]:
def get_file_with_id(id):
    index = id // (n // len(test_files))
    path = test_files_path.replace("*", "_{}".format(index))
    try:
        return test_files.index(path)
    except ValueError:
        return

get_file_with_id(884)

4446

In [101]:
for i, test in enumerate(tests):
    if len(test[test["isFraud"] == 1]) > 60:
        print(i)

16098
16099
16100
16101
16102
16103
16104
16555
16556
87319
99996
99997
99998
99999


In [102]:
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import VectorAssembler 

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

training_dir = "/Users/joaoneves/Documents/demo-iot-transactions/data/models"
model_path = os.path.join(training_dir, "pythonLogisticRegression")

In [103]:
data = spark.read.csv(training_path, header=True)

# data.printSchema()
columns = data.columns
float_columns = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'gps_latitude', 'gps_longitude']
target_column = 'isFraud'
print(columns)
data.take(4)

['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud', 'gps_latitude', 'gps_longitude', 'id']


[Row(step='1', type='PAYMENT', amount='9839.64', nameOrig='C1231006815', oldbalanceOrg='170136.0', newbalanceOrig='160296.36', nameDest='M1979787155', oldbalanceDest='0.0', newbalanceDest='0.0', isFraud='0', isFlaggedFraud='0', gps_latitude='51.50256769719365', gps_longitude='-0.12312710200156707', id='0'),
 Row(step='1', type='PAYMENT', amount='1864.28', nameOrig='C1666544295', oldbalanceOrg='21249.0', newbalanceOrig='19384.72', nameDest='M2044282225', oldbalanceDest='0.0', newbalanceDest='0.0', isFraud='0', isFlaggedFraud='0', gps_latitude='51.457455017283436', gps_longitude='-0.10674969663788776', id='1'),
 Row(step='1', type='TRANSFER', amount='181.0', nameOrig='C1305486145', oldbalanceOrg='181.0', newbalanceOrig='0.0', nameDest='C553264065', oldbalanceDest='0.0', newbalanceDest='0.0', isFraud='1', isFlaggedFraud='0', gps_latitude='51.467931861733966', gps_longitude='-0.11230149709138582', id='2'),
 Row(step='1', type='CASH_OUT', amount='181.0', nameOrig='C840083671', oldbalanceOrg

In [104]:
def parse_row(row):
    new_row = {}
    for column in columns:
        if column in float_columns:
            new_row[column] = float(row[column])
        elif column == target_column:
            new_row["label"] = int(round(float(row[column])))
        else:
            new_row[column] = row[column]
    return Row(**new_row)

data_parsed = data.rdd.map(parse_row).toDF()

data_parsed.head()

Row(amount=9839.64, gps_latitude=51.50256769719365, gps_longitude=-0.12312710200156707, id='0', isFlaggedFraud='0', label=0, nameDest='M1979787155', nameOrig='C1231006815', newbalanceDest=0.0, newbalanceOrig=160296.36, oldbalanceDest=0.0, oldbalanceOrg=170136.0, step=1.0, type='PAYMENT')

In [105]:
vecAssembler = VectorAssembler(inputCols=['step','amount', \
                                          'oldbalanceOrg', 'newbalanceOrig', \
                                          'oldbalanceDest', 'newbalanceDest', \
                                          'gps_latitude', 'gps_longitude'], \
                               outputCol="features")

transformed_data = vecAssembler.transform(data_parsed)

transformed_data.head()

Row(amount=9839.64, gps_latitude=51.50256769719365, gps_longitude=-0.12312710200156707, id='0', isFlaggedFraud='0', label=0, nameDest='M1979787155', nameOrig='C1231006815', newbalanceDest=0.0, newbalanceOrig=160296.36, oldbalanceDest=0.0, oldbalanceOrg=170136.0, step=1.0, type='PAYMENT', features=DenseVector([1.0, 9839.64, 170136.0, 160296.36, 0.0, 0.0, 51.5026, -0.1231]))

In [106]:
lr = LogisticRegression(maxIter=1000, threshold=0.01, tol=1e-5)

model = lr.fit(transformed_data)

In [107]:
model.evaluate(transformed_data).pr.show()
print(model.evaluate(transformed_data).areaUnderROC)

+------------------+--------------------+
|            recall|           precision|
+------------------+--------------------+
|               0.0|                 1.0|
| 0.774869109947644| 0.09813415574402468|
| 0.816997443078047|0.052232938667164865|
| 0.843784244490442| 0.03607946854370145|
|0.8671618166321685|0.027854150941182453|
|0.8818945574089857|0.022683854156879693|
|0.8892000487032753|0.019072117832939425|
|0.8929745525386582|0.016424501934929052|
|0.8988189455740898|0.014470509133749298|
|0.9025934494094728|0.012920171362639737|
|0.9075855351272373|0.011694988915334494|
|0.9109947643979057|0.010673628348882993|
|0.9137952027273835|0.009967050961445243|
|0.9178132229392426| 0.00923102545450983|
| 0.920370144892244|0.008587752653637155|
|0.9237793741629124| 0.00803860905681168|
|0.9262145379276756|0.007550874350949393|
|0.9290149762571533|0.007123877032359021|
|0.9325459637160599|0.006750045608416273|
|0.9347376111043467| 0.00640672568498638|
+------------------+--------------

In [108]:
model.evaluate(transformed_data).recallByThreshold.show()

+--------------------+------------------+
|           threshold|            recall|
+--------------------+------------------+
|                 1.0| 0.774869109947644|
|0.006561491282644194| 0.816997443078047|
|0.003949357813573477| 0.843784244490442|
|0.002835492888867661|0.8671618166321685|
|0.002107414662537...|0.8818945574089857|
|0.001733748409879...|0.8892000487032753|
|0.001544655807152...|0.8929745525386582|
|0.001444811147117...|0.8988189455740898|
|0.001352713724098...|0.9025934494094728|
|0.001277444178936144|0.9075855351272373|
|0.001201056567940...|0.9109947643979057|
|0.001135738789277...|0.9117253135273348|
|0.001115631623257...|0.9163521246803847|
|0.001053026804150195|0.9192743211981006|
|9.946928734003158E-4|0.9218312431511019|
| 9.42524646181952E-4|0.9245099232923414|
|8.910491138542175E-4|0.9277973943747717|
|8.474195713039216E-4|0.9312066236454402|
|8.056726289204571E-4|0.9331547546572507|
|7.664274069040154E-4|0.9359551929867284|
+--------------------+------------

In [109]:
# Save and load model
model.write().overwrite().save(model_path)

In [110]:
sameModel = LogisticRegressionModel.load(model_path)

In [111]:
pred = sameModel.transform(transformed_data)

In [113]:
anomalies = pred.rdd.filter(lambda row: row["prediction"]==1.0)

In [116]:
len(tests[2])

64

In [165]:
transactions_ids = anomalies.map(lambda row: (get_file_with_id(int(row["id"])), row["id"])).groupByKey().mapValues(lambda l: list(l)).cache()

In [167]:
transactions_ids.take(10)

[(11112, ['146', '165']),
 (12224, ['1362']),
 (22224, ['1898', '1904']),
 (31112, ['2417']),
 (51112, ['3569']),
 (4224, ['8706']),
 (5224, ['9284']),
 (7224, ['10395', '10396']),
 (11336, ['12787']),
 (12336, ['13327', '13333', '13336'])]

In [132]:
6362620 // 100000

63

In [135]:
884 // 63

14